In [ ]:
# %matplotlib notebook
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
from time_domain_astronomy_sandbox.backend import Backend
from time_domain_astronomy_sandbox.observation import Observation
from time_domain_astronomy_sandbox.pulse import Pulse
from time_domain_astronomy_sandbox.plotting import plot_multi_images

import numpy as np
from matplotlib import rc

# Dispersed pulse covering all of ARTS band

In [ ]:
def interactive_pulse_arts():
    """Plot interactive dispersed pulse using ASTRON's ARTS backend."""
    pulse = Pulse(Backend())
    pulse.plot_delay_v_frequency_interactive(xscale='linear')

interactive_pulse_arts()

# Dispersed pulse from ARTS band to Lofar HBA band

In [ ]:
def interactive_pulse_arts_to_lofar_hba():
    """Plot interactive dispersed pulse using bands from ARTS to Lofar HBA."""
    fmin = 120
    n_channels = 3000
    channel_bandwidth = (Backend().fmax-fmin)/n_channels

    pulse = Pulse(
        Backend(
            fmin=fmin,
            n_channels=n_channels,
            channel_bandwidth=channel_bandwidth
            )
        )
    pulse.plot_delay_v_frequency_interactive(xscale='linear', dm_init=3000)

interactive_pulse_arts_to_lofar_hba()

# Add one dispersed pulse, and dedisperse it.

In [ ]:
def test_one_pulse_dedisperse():
    """Plot one dispersed pulse, and it's dedispersed self."""
    obs = Observation(Backend(), length=1.5*1.024)
    obs_original = obs.window.copy()
    obs.add_dispersed_pulse(dm=1000, width=0.006, pulse_t0=0.16, snr=12, verbose=True)
    xstep = 1000
    ystep = 500

    rc('font', size=12)
    rc('axes', titlesize=12)
    rc('axes', labelsize=12)

    plot_multi_images(
        (
            obs_original,
            obs.window,
            obs.dedisperse(dm=1000),
        ),
        direction='vertical',

        xticks=obs.time_indices[::xstep],
        xtick_labels=["%.2f" % t for t in obs.times[::xstep]],

        yticks=obs.backend.freq_indices[::ystep],
        ytick_labels=["%.0f" % f for f in obs.backend.frequencies[::ystep]],

        xfig_size = 30,
        yfig_size = 10,
        spectrum=False,    # experimental viz
        
        savefig=True,
        fig_name='dm_1000_width_0.006s_t0_0.16s_snr_12',
        ext='png'
    )
    
    print(obs.window.min(), obs.window.max(), obs.window.mean(), obs.window.std())

test_one_pulse_dedisperse()

# Add 1 low DM pulse, 1 high DM Dispersed pulses, and RFI 
## + RFI cleaning

In [ ]:
def pulse_and_rfi__cleaned(spectrum=False):
    """Plot pulses and rfi, and rfi clean.

    Plot DM=1000 Width=6ms S/N=12
    Plot DM=1 Width=6ms S/N=50
    Plot DM=10 Width=6ms S/N=15
    Plot RFI < 32 channels wide

    Property
    --------
    spectrum : bool
    """
    obs = Observation(Backend(), length=1.024)
    raw = obs.window.copy()
    obs.add_dispersed_pulse(dm=1000, width=0.006, pulse_t0=0.16, snr=12)
    obs.add_dispersed_pulse(dm=1, width=0.006, pulse_t0=0.33, snr=18)
#     obs.add_dispersed_pulse(dm=10, width=0.006, pulse_t0=0.73, snr=17)

    for f1, f2 in [[64, 73], [560, 575], [1220, 1255]]:
        obs.add_rfi(
            t_start=0.,
            t_stop=0.5,
            t_step=0.03,
            f_start=f1, f_stop=f2,
            width=0.01,
            snr=20,
        )

    xstep = 1000
    ystep = 500

    rc('font', size=12)
    rc('axes', titlesize=12)
    rc('axes', labelsize=12)

    o_window = obs.window.copy()
    o_tc = obs.time_cleaning().copy()
    o_fc = obs.frequency_cleaning().copy()

    plot_multi_images(
        (
            raw,
            o_window,
            o_tc,
            o_fc,
            obs.frequency_cleaning(obs.time_cleaning(), keep_state=True),
            obs.frequency_cleaning(obs.time_cleaning(), keep_state=True),
            obs.frequency_cleaning(obs.time_cleaning(), keep_state=True),
            obs.dedisperse(dm=1000),
        ),
        direction='vertical',

        xticks=obs.time_indices[::xstep],
        xtick_labels=["%.2f" % t for t in obs.times[::xstep]],

        yticks=obs.backend.freq_indices[::ystep],
        ytick_labels=["%.0f" % f for f in obs.backend.frequencies[::ystep]],

        xfig_size=50,
        yfig_size=20,
        spectrum=spectrum,
        savefig=True,
        fig_name='pulses_rfi_rficlean',
        ext='png'
    )
    
pulse_and_rfi__cleaned(spectrum=False)